In [1]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI, MCSAPI

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("SWAN_MAIN_API"), environment=os.getenv("SWAN_ENVIRONMENT"))

api_key = os.getenv("MCS_API")
mcs_api = MCSAPI(api_key)


In [2]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['North Rhine-Westphalia-DE',
   'Kyiv City-UA',
   'England-GB',
   'Guangdong-CN',
   'Kowloon-HK',
   'Jiangsu-CN',
   'Ivano-Frankivsk Oblast-UA',
   'Beijing-CN',
   'Henan-CN',
   'Saxony-DE',
   'North West-SG',
   'Illinois-US',
   'Kwai Tsing-HK',
   'North Carolina-US',
   'Quebec-CA',
   'California-US',
   'Bavaria-DE',
   'Central and Western District-HK',
   'Tokyo-JP',
   'Bashkortostan Republic-RU',
   'Jeollanam-do-KR',
   'Kowloon City-HK',
   'North Holland-NL'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['North Rhine-Westphalia-DE',
   'Guangdong-CN',
   'Kowloon-HK',
   'Jiangsu-CN',
   'Ivano-Frankivsk Oblast-UA',
   'Beijing-CN',
   'Henan-CN',
   'North West-SG',
   'Illinois-US',
   'Kwai Tsing-HK',
   'North Carolina-US',
   'Quebec-CA',
 

In [3]:
device = "G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

13
('G1ae.medium', 13, ['Kwai Tsing-HK', 'Kowloon City-HK', 'Quebec-CA', 'Guangdong-CN', 'California-US', 'Tokyo-JP'])


In [4]:
from swan.object.source_uri import LagrangeSpace

lag = LagrangeSpace(os.getenv("WALLET_ADDRESS"), 'NSFW', os.getenv("WALLET_ADDRESS"), obj)
 
lag.get_space_info()

# Upload Directory to MCS
response = lag.generate_source_uri('swan_test', 'mar8s1', 'nsfw.json', mcs_client = mcs_api)

In [5]:
print(lag.source_uri)

https://plutotest.acl.swanipfs.com/ipfs/QmaGAPK7kSxDdoFAaKmi9Rn89QApVGxTgBrT64exzpWE1x


In [6]:
import sys
sys.path.insert(0, '..')
 
import os
from dotenv import load_dotenv
load_dotenv()
 
from swan import SwanContract
 
pk = os.getenv('Pri_Key')
rpc = os.getenv('RPC')
 
c = SwanContract(pk, rpc)
 
# Test esimate lock revenue
r = c.estimate_payment(obj.id, 2)
print(r)

22000000000000000000


In [7]:
r = c._approve_swan_token(22000000000000000000)
print(r)
 
r = c.lock_revenue('1', obj.id, 1)
print(r)

0xff295da94aa7cb6ef9061935332904e90568fd68a08e69d553acd2b649224f69
0xfe9ea3518c85e2542f4e1ba11dde6204f0259de00d92f2f7227a1c8267f4a999


In [8]:
# Retrieve List of Hardwares

# for hardware in hardwares:
#     if hardware.name == 'C1ae.medium':
#         print(hardware.region, hardware.price)
        
# Deploy task
result = swan_api.deploy_task(
                              cfg_name=device, 
                              region='Quebec-CA', 
                              start_in=60*40, duration=3600*1, 
                              job_source_uri="https://plutotest.acl.swanipfs.com/ipfs/QmPA19sHXnBPHWk2oohQZtogV4zDc8GZPvsv97xQP3eJSj", 
                              paid=11,
                              tx_hash="0xfe9ea3518c85e2542f4e1ba11dde6204f0259de00d92f2f7227a1c8267f4a999",
                              wallet_address=os.getenv('WALLET_ADDRESS'),
                            )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

{'data': {'task': {'created_at': '1711827396', 'end_at': '1711833389', 'leading_job_id': None, 'status': 'created', 'task_detail_cid': 'https://data.mcs.lagrangedao.org/ipfs/QmeqRa2HzEsex7UwLuY82P49eDTR29pk56wHCMJJw3461A', 'updated_at': '1711827396', 'uuid': '3b10df76-551f-40d2-be3e-8cd32ad023b5'}}, 'message': 'Task_uuid created.', 'status': 'success'}
Task UUID: 3b10df76-551f-40d2-be3e-8cd32ad023b5


In [18]:
import json

# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_uri = info['data']['jobs'][0]['job_result_uri']
        job_real_uri = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_uri)
        print("Job Real URL: ", job_real_uri)
        
        # break
        if status == 'deploytok8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(30)

buildImage
Job Result URL:  https://42f6d9f62851.acl.swanipfs.com/ipfs/QmYFdA8s34gfvHRuZgZKiVLu4zXiBWfjZ95arLjzKAWJTZ
Job Real URL:  https://6xlq0epd3i.filezoo.com.cn


KeyboardInterrupt: 

In [10]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://6xlq0epd3i.filezoo.com.cn']


In [33]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}
json_data = {
    'link': [
        'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s',
        'https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png',
        'www.google.com',
        'www.pornhub.com',
    ],
}
print("Job real uri: ", os.getenv('JOB_REAL_URI'))

response = requests.post(f"{os.getenv('JOB_REAL_URI')}/link", headers=headers, json=json_data)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


Job real uri:  https://r31khailwj.dev2.crosschain.computer
{
    "result": [
        {
            "is_nsfw_image": true,
            "is_nsfw_link": false,
            "link": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s",
            "probability": 0.999750554561615
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png",
            "probability": null
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "www.google.com",
            "probability": null
        },
        {
            "is_nsfw_image": null,
            "is_nsfw_link": true,
            "link": "www.pornhub.com",
            "probability": 100
        }
    ]
}


In [30]:
log_path = '../nsfw_detection.csv'

if not os.path.isfile(log_path):
    with open(log_path, 'w') as f:
        f.write('link,IS NSFW IMAGE,IS NSFW LINK,Probability\n')

with open(log_path, 'a') as f:
    for result in response.json()['result']:
        f.write(f"{result['link']},{result['is_nsfw_image']},{result['is_nsfw_link']},{result['probability']}\n")
